# Plot neutralization curves

In this notebook we will create neutralization plots from GFP-based neutralization assays. 
The GFP-based neutralization assay system is described in detail in [Bloom et al (2010)](http://science.sciencemag.org/content/328/5983/1272.long) and [Hooper and Bloom (2013)](https://jvi.asm.org/content/87/23/12531).

In brief, we generate by reverse genetics influenza viruses that carry GFP in the PB1 segment. We then quantified using a plate reader the GFP signal produced by MDCK-SIAT1-CMV-PB1 cells infected with PB1flank-eGFP virus that had been incubated with various dilutions of antibody or serum. The neutralization curves represent the mean and standard deviation of three measurements, with each replicate in a separate column of a 96-well plate.

In [ ]:
import os
import subprocess
import glob
import math
import time
import pandas as pd
import numpy as np
from colour import Color
from scipy.optimize import curve_fit

import matplotlib
import matplotlib.lines as mlines
matplotlib.use("Pdf")
print ('Using matplotlib version {0}'.format(matplotlib.__version__))
from matplotlib.ticker import MaxNLocator
import pylab as plt
from IPython.display import Image, display

import dms_tools2
from dms_tools2.ipython_utils import showPDF

plotsdir = './plots/'
if not os.path.isdir(plotsdir):
    os.mkdir(plotsdir)

In [ ]:
########################
# Read in neutralization data csv files
########################

# Monoclonal antibodies
mAb5A01 = pd.read_csv('./data/mAb_5A01.csv')
mAb3C06 = pd.read_csv('./data/mAb_3C06.csv')
mAb3C04 = pd.read_csv('./data/mAb_3C04.csv')
mAb4C01 = pd.read_csv('./data/mAb_4C01.csv')
mAb1C04 = pd.read_csv('./data/mAb_1C04.csv')
mAb4F03 = pd.read_csv('./data/mAb_4F03.csv')

# 2015/2016 pre- and post-vaccination human serum samples
# From Hensley lab, Penn
SH_589 = pd.read_csv('./data/SH_589.csv')
SH_557 = pd.read_csv('./data/SH_557.csv')
SH_571 = pd.read_csv('./data/SH_571.csv')
SH_574 = pd.read_csv('./data/SH_574.csv')

# Human serum samples collected 2008-2010
# From VIDD, Fred Hutch
VIDD_F64104 = pd.read_csv('./data/VIDD_F64104.csv')
VIDD_H58373 = pd.read_csv('./data/VIDD_H58373.csv')
VIDD_W41207 = pd.read_csv('./data/VIDD_W41207.csv')
VIDD_H51589 = pd.read_csv('./data/VIDD_H51589.csv')
VIDD_W76735 = pd.read_csv('./data/VIDD_W76735.csv') # collected 2 months after H51589

# Ferret serum samples pre- and post-infection with Perth/2009 with our HA
# From Lakdawala lab, Pittsburgh
f9267 = pd.read_csv('./data/ferret_Pitt_9267.csv')
f9435 = pd.read_csv('./data/ferret_Pitt_9435.csv')
f9437 = pd.read_csv('./data/ferret_Pitt_9437.csv')

# Ferret serum samples from post-infection
# From Aeron Hurt, WHO CC, Melbourne
fWHOCC_Perth2009 = pd.read_csv('./data/ferret_WHOCC_Perth2009.csv')
fWHOCC_Vic11 = pd.read_csv('./data/ferret_WHOCC_Vic11.csv')

In [ ]:
# Define functions to fit curves to neutralization assay data
def sigmoid(x, x0, k):
    '''from example use of scipy.optimize.curve_fit found at https://gist.github.com/andrewgiessel/5684769'''
    # forces 0% infectivity
    y = 1 / (1 + np.exp(-k*(x-x0)))
    return y

def sigmoid2(x, x0, k, a):
    '''from example use of scipy.optimize.curve_fit found at https://gist.github.com/andrewgiessel/5684769'''
    # additional parameter for bottoming out above or below zero
    y = 1 / (1 + np.exp(-k*(x-x0)))+a
    return y

def expo(x, a, b, c):
    y = a*np.exp(-b*x)-c
    return y

def expo_fixed_topbottom(x, b):
    '''fixes max infectivity to 1, min infectivity to 0'''
    y = 1*np.exp(-b*x)
    return y


# Define a function to create the neutralization curve plot
def PlotNeutralizationCurves(data, x_datalabel, y_datalabels, outfile, 
                             color_cycle=['k','k','k','k','k','k'],
                             style_cycle=['solid','solid','solid','solid','solid','solid'],
                             fit_cycle=[sigmoid,sigmoid,sigmoid,sigmoid,sigmoid,sigmoid],
                             fmt_cycle = ['o','o','o','o','o','o'],
                             linewidth=2, fit_curve = False, x_lim = None, y_lim = None, figsize=(5,4),
                             verbose=False, x_label_override=False, alpha=1, markersize=9, mew=2,
                             legend_placement_override=False, no_legend=False,
                             legendfontsize=14, vlines = None):
    '''
    *data* is a dataframe of mAb concentrations and resulting fraction infectivity/stdev. 
    The mAb concentrations have column header *x_datalabel*, and the 
    fractional infectivity and stdev of fractional infectivity should have column headers of
    the items of the list *y_datalabels* followed by '_avg' and '_std', respectively.
    '''
        
    f, ax = plt.subplots()
    handles = [] # proxy artists for the legend to show a single legend entry representing the data (marker) and the curve fit (line)
    
    for i,y_datalabel in enumerate(y_datalabels):
        if fit_curve:
            try: 
                popt, pcov = curve_fit(fit_cycle[i], data[x_datalabel], data[y_datalabel + '_avg'])
                minlog = math.log10(min(data[x_datalabel]))
                maxlog = math.log10(max(data[x_datalabel]))
                xcurve = np.logspace(minlog, maxlog, num=100)
                ycurve = fit_cycle[i](xcurve, *popt)
                ax.plot(xcurve, ycurve, color=color_cycle[i], linewidth=linewidth, linestyle=style_cycle[i], label=y_datalabel, alpha=alpha)
                
                if verbose:
                    print ("\n\ncurve-fitting info for ", y_datalabel)
                    print ("fit parameters for curve: ", popt)
                    if fit_cycle[i] == expo:
                        (a, b, c) = popt
                        ic50 = np.log((0.5+c)/a)/-b
                        print ("expo fit curve was used; here is the IC50 calculated from the fit parameters: ", ic50)
                    elif fit_cycle[i] == expo_fixed_top:
                        (b, c) = popt
                        ic50 = np.log((0.5+c)/1.0)/-b
                        print ("expo fit fixed top curve was used; here is the IC50 calculated from the fit parameters: ", ic50)
            except:
                print ("Threw an exception during fitting {0} curve for {1}".format(fit_cycle[i], y_datalabel))
                        
        ax.errorbar(data[x_datalabel], data[y_datalabel + '_avg'], 
                    yerr=data[y_datalabel + '_std'], 
                    color=color_cycle[i], fmt=fmt_cycle[i], 
                    capthick=2, elinewidth=2, markersize=markersize, capsize=markersize/1.5,
                    label=y_datalabel, mew=mew)
        
        custom_handle = mlines.Line2D([], [], color=color_cycle[i], marker=fmt_cycle[i], 
                                      markersize=1.*markersize, linestyle=style_cycle[i], label=y_datalabel,
                                      linewidth=1.*linewidth, mew=mew)
        handles.append(custom_handle)
    
    if x_lim:
        ax.set_xlim(x_lim)
    else:
        ax.set_xlim([4E-3,100])
    ax.set_xscale("log")
    
    if y_lim:
        ax.set_ylim(y_lim)
    else:
        ax.set_ylim([-0.05,1.2])
        
    plt.yticks([0.00, 0.25, 0.5, 0.75, 1.00], ['0', '0.25', '0.50', '0.75', '1.0'])
    
    spineOffset = {'left': 8, 'bottom': 8}    
    [spine.set_position(('outward',spineOffset[loc])) if loc in ['left','bottom'] else spine.set_color('none') for loc, spine in ax.spines.items() ] 
    plt.rcParams['xtick.direction'] = 'out'
    plt.rcParams['ytick.direction'] = 'out'
    ax.xaxis.set(ticks_position = 'bottom')
    ax.yaxis.set(ticks_position = 'left')
    plt.tick_params(axis='both', which='major', labelsize=16, width=2)
    
    #ax.minorticks_off()
    
    if x_label_override:
        plt.xlabel(x_label_override, fontsize=19)
    else:
        plt.xlabel('$\mu$g/ml', fontsize=19)

    plt.ylabel('fraction infectivity', fontsize=19)
    
    if vlines:
        for xc in vlines:
            plt.axvline(x=xc, linestyle = 'dashed', linewidth=3, color='#CC79A7', dashes=(2, 3))
    
    if not no_legend:
        if legend_placement_override:
            lgd = ax.legend(handles, y_datalabels, fontsize=legendfontsize, 
                        loc=legend_placement_override, 
                        numpoints = 1, markerscale=1.)
        else:
            lgd = ax.legend(handles, y_datalabels, fontsize=legendfontsize, 
                            bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., 
                            numpoints = 1, markerscale=1.)
        f.set_size_inches(figsize)
        plt.savefig(outfile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    
    else:
        f.set_size_inches(figsize)
        plt.savefig(outfile, bbox_inches='tight')

    plt.close()

## Plot neutralization curves for monoclonal antibodies

In [ ]:
#Plotting neutralization curve for 5A01
mut = ['WT']
color_cycle = ['k']
style_cycle = ['solid']
fit_cycle = [expo]
fmt_cycle = ['o']

PlotNeutralizationCurves(mAb5A01, 'ug/ml_5A01', 
                         mut,
                         './plots/mAb_5A01.pdf', 
                         color_cycle=color_cycle,
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-4,5], y_lim = [-0.1,1.2], 
                         x_label_override='$\mu$g/ml')

showPDF('./plots/mAb_5A01.pdf')

In [ ]:
#Plotting neutralization curve for 3C06
mut = ['WT']
color_cycle = ['k']
style_cycle = ['solid']
fit_cycle = [expo]
fmt_cycle = ['o']

PlotNeutralizationCurves(mAb3C06, 'ug/ml_3C06', 
                         mut,
                         './plots/mAb_3C06.pdf', 
                         color_cycle=color_cycle,
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-4,5], y_lim = [-0.1,1.2], 
                         x_label_override='$\mu$g/ml')

showPDF('./plots/mAb_3C06.pdf')

In [ ]:
#Plotting neutralization curve for 3C04
mut = ['WT']
color_cycle = ['k']
style_cycle = ['solid']
fit_cycle = [expo]
fmt_cycle = ['o']

PlotNeutralizationCurves(mAb3C04, 'ug/ml_3C04', 
                         mut,
                         './plots/mAb_3C04.pdf', 
                         color_cycle=color_cycle,
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-4,5], y_lim = [-0.1,1.2], 
                         x_label_override='$\mu$g/ml')

showPDF('./plots/mAb_3C04.pdf')

In [ ]:
#Plotting neutralization curve for 4C01
mut = ['WT']
color_cycle = ['k']
style_cycle = ['solid']
fit_cycle = [expo]
fmt_cycle = ['o']

PlotNeutralizationCurves(mAb4C01, 'ug/ml_4C01', 
                         mut,
                         './plots/mAb_4C01.pdf', 
                         color_cycle=color_cycle,
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-4,5], y_lim = [-0.1,1.2], 
                         x_label_override='$\mu$g/ml')

showPDF('./plots/mAb_4C01.pdf')

In [ ]:
#Plotting neutralization curve for 1C04
mut = ['WT']
color_cycle = ['k']
style_cycle = ['solid']
fit_cycle = [expo]
fmt_cycle = ['o']

PlotNeutralizationCurves(mAb1C04, 'ug/ml_1C04', 
                         mut,
                         './plots/mAb_1C04.pdf', 
                         color_cycle=color_cycle,
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-4,30], y_lim = [-0.1,1.2], 
                         x_label_override='$\mu$g/ml')

showPDF('./plots/mAb_1C04.pdf')

In [ ]:
#Plotting neutralization curve for 4F03
mut = ['WT']
color_cycle = ['k']
style_cycle = ['solid']
fit_cycle = [expo]
fmt_cycle = ['o']

PlotNeutralizationCurves(mAb4F03, 'ug/ml_4F03', 
                         mut,
                         './plots/mAb_4F03.pdf', 
                         color_cycle=color_cycle,
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-4,30], y_lim = [-0.1,1.2], 
                         x_label_override='$\mu$g/ml')

showPDF('./plots/mAb_4F03.pdf')

## Plot neutralization curves for 2015/2016 pre- and post-vaccination human serum samples

In [ ]:
color1 = '#E69F00' # gold
color2 = '#56B4E9' # blue
color3 = '#009E73' # green

In [ ]:
mut = ['589pre-vacc', '589post-vacc']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo, expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(SH_589, 'serum_dilution', 
                         mut,
                         './plots/SH_589.pdf', 
                         color_cycle=[color1, 'k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/SH_589.pdf')

In [ ]:
mut = ['557pre-vacc', '557post-vacc']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo, expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(SH_557, 'serum_dilution', 
                         mut,
                         './plots/SH_557.pdf', 
                         color_cycle=[color1, 'k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/SH_557.pdf')

In [ ]:
mut = ['571pre-vacc', '571post-vacc']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo, expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(SH_571, 'serum_dilution', 
                         mut,
                         './plots/SH_571.pdf', 
                         color_cycle=[color1, 'k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/SH_571.pdf')

In [ ]:
mut = ['574pre-vacc', '574post-vacc']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo, expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(SH_574, 'serum_dilution', 
                         mut,
                         './plots/SH_574.pdf', 
                         color_cycle=[color1, 'k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/SH_574.pdf')

## Plot neutralization curves for VIDD human serum samples 

In [ ]:
mut = ['WT']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(VIDD_F64104, 'serum_dilution', 
                         mut,
                         './plots/VIDD_F64104.pdf', 
                         color_cycle=['k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/VIDD_F64104.pdf')

In [ ]:
mut = ['WT']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(VIDD_H58373, 'serum_dilution', 
                         mut,
                         './plots/VIDD_H58373.pdf', 
                         color_cycle=['k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/VIDD_H58373.pdf')

In [ ]:
mut = ['WT']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(VIDD_W41207, 'serum_dilution', 
                         mut,
                         './plots/VIDD_W41207.pdf', 
                         color_cycle=['k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/VIDD_W41207.pdf')

In [ ]:
mut = ['WT']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(VIDD_H51589, 'serum_dilution', 
                         mut,
                         './plots/VIDD_H51589.pdf', 
                         color_cycle=['k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/VIDD_H51589.pdf')

In [ ]:
mut = ['WT']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(VIDD_W76735, 'serum_dilution', 
                         mut,
                         './plots/VIDD_W76735.pdf', 
                         color_cycle=['k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/VIDD_W76735.pdf')

## Plot neutralization curves for ferret serum samples 

In [ ]:
mut = ['f9267neg', 'f9267post-infection']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [sigmoid, expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(f9267, 'serum_dilution', 
                         mut,
                         './plots/ferret_Pitt_9267.pdf', 
                         color_cycle=[color1, 'k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/ferret_Pitt_9267.pdf')

In [ ]:
mut = ['f9435neg', 'f9435post-infection']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [sigmoid, expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(f9435, 'serum_dilution', 
                         mut,
                         './plots/ferret_Pitt_9435.pdf', 
                         color_cycle=[color1, 'k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/ferret_Pitt_9435.pdf')

In [ ]:
mut = ['f9437neg', 'f9437post-infection']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [sigmoid, expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(f9437, 'serum_dilution', 
                         mut,
                         './plots/ferret_Pitt_9437.pdf', 
                         color_cycle=[color1, 'k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/ferret_Pitt_9437.pdf')

In [ ]:
mut = ['WHOCC']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo_fixed_topbottom]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(fWHOCC_Perth2009, 'serum_dilution', 
                         mut,
                         './plots/ferret_WHOCC_Perth2009.pdf', 
                         color_cycle=['k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/ferret_WHOCC_Perth2009.pdf')

In [ ]:
mut = ['WHOCCVic']
style_cycle = ['solid', 'solid', 'solid', 'solid']
fit_cycle = [expo]
fmt_cycle = ['^','o']

PlotNeutralizationCurves(fWHOCC_Vic11, 'serum_dilution', 
                         mut,
                         './plots/ferret_WHOCC_Vic11.pdf', 
                         color_cycle=['k'],
                         style_cycle=style_cycle,
                         fit_cycle=fit_cycle,
                         fmt_cycle=fmt_cycle,
                         linewidth = 2.5,
                         fit_curve = True, 
                         verbose = True,
                         x_lim =[1E-5,0.5], y_lim = [-0.1,1.2], 
                         x_label_override='serum dilution')

showPDF('./plots/ferret_WHOCC_Vic11.pdf')